In [ ]:
import requests
import json
from csv import reader 
import os
import sys
from bs4 import BeautifulSoup
import time
import random
from lxml import etree
import os
import re
import datetime
# # from selenium import webdriver


In [ ]:
nowTime = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [ ]:
# year = ['2014','2015','2016','2017','2018','2019','2020','2021','2022']
domain = 'https://www.caranddriver.com'
headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Host':'www.kbb.com',
        'Referer':'https://www.kbb.com/car-reviews/',
        'Content-Type':'application/json',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
        }

In [ ]:
response = requests.get(url=domain)
soup= BeautifulSoup(response.text, "lxml")
soup.select('option[class="css-0 e1ofnlcl0"]')

In [ ]:
make=[
# 'Acura',
# 'AlfaRomeo',
# 'AstonMartin',
# 'Audi',
# 'Bentley',
# 'BMW',
# 'Bollinger',
# 'Bugatti',
# 'Buick',
# 'Byton',
# 'Cadillac',
# 'Chevrolet',
# 'Chrysler',
# 'Dodge',
# 'Ferrari',
'Fiat'#,
# 'Fisker',
# 'Ford',
# 'Genesis',
# 'GMC',
# 'Honda',
# 'Hummer',
# 'Hyundai',
# 'Infiniti',
# 'Jaguar',
# 'Jeep',
# 'Karma',
# 'Kia',
# 'Koenigsegg',
# 'Lamborghini',
# 'LandRover',
# 'Lexus',
# 'Lincoln',
# 'Lordstown',
# 'Lotus',
# 'LucidMotors',
# 'Maserati',
# 'Maybach',
# 'Mazda',
# 'McLaren',
# 'Mercedes-AMG',
# 'Mercedes-Benz',
# 'Mercedes-Maybach',
# 'Mercury',
# 'Mini',
# 'Mitsubishi',
# 'Nikola',
# 'Nissan',
# 'Pagani',
# 'Polestar',
# 'Pontiac',
# 'Porsche',
# 'Ram',
# 'Rivian',
# 'Rolls-Royce',
# 'Saab',
# 'Saturn',
# 'Scion',
# 'Scout',
# 'Smart',
# 'Spyker',
# 'Subaru',
# 'Suzuki',
# 'Tesla',
# 'Toyota',
# 'VinFast',
# 'Volkswagen',
# 'Volvo'
]

In [ ]:
for m in make:
    url_main=domain+'/'+m.lower()
    # print(url_main)
    response = requests.get(url=url_main)
    soup=soup= BeautifulSoup(response.text, "lxml")
    href=soup.select('a[class="vehicle-item-title item-title"]')
    for h in href:
        url_each=h.get('href')
        print(domain+url_each)

In [63]:
url='https://www.caranddriver.com/fiat/500x'
response = requests.get(url=url)
soup= BeautifulSoup(response.text, "lxml")

In [64]:
vehicle={}
try:
    vehicle={
    'name':soup.select('h1[class="css-i4j13t e10ise8i2"]')[0].text,
    'url':soup.select('#main-content > header > div.css-16uv341.endngjl0 > div.css-70qvj9.e17a0kax2 > div > a[title="Specs"]')[0].attrs['href']
    }
except IndexError:
    pass

In [85]:
str=soup.select('h1[class="css-i4j13t e10ise8i2"]')[0].text
year = re.findall('(^\d{4})',str)[0]
year

'2022'

In [86]:
str='2022 Fiat 500X'
last=re.findall('[ ](.*)',str)[0].replace(' ','/').lower()
url_back=last+'/'+year

In [88]:
url='https://www.kbb.com'+'/'+url_back
url

'https://www.kbb.com/fiat/500x/2022'

In [89]:
response = requests.get(url=url)